In [ ]:
# =============================================================================
# zeta_buildings_typeC.ipynb
# =============================================================================
# This script computes explicit spectral data for type C_n buildings,
# producing per–(λ,μ) contributions to the zeta factor
#   (1 ± q^m u^c)^[n_χ]*[d_{λ,μ}(q)].
#
# Main components:
#  • generic_degree() – Carter–Geck–Pfeiffer hook–cohook degree d_{λ,μ}(q)
#  • multiplicities_bipartitions_new() – multiplicities n_χ via Kostka sums
#  • reflection_statistic_f() – reflection sum f_χ
#  • col7_*() – special and generic character values χ^{λ,μ}(u e_{W_I})
#  • table_rows_for_new() – build rows with all invariants
#  • list_factors_per_row() / pretty_print_factors_per_row() – display factors
#
# Generic case (i≠j): (1 − (−1)^{|μ|} q^{(f−2ℓ)/2} u⁴)^[n_χ]*[d_χ]
# Special case (i=j): split into (1±q^{(f−2ℓ)/4}u²)^[ (n_χ±col7)/2 ]*[d_χ].
# =============================================================================



from sage.all import Partitions, Partition, SymmetricFunctions, QQ, binomial, ZZ, var

# --- Base ring: polynomials in q (not the fraction field)
R.<q> = QQ[]               # polynomial ring in q

# ---- Symmetric functions setup
SF = SymmetricFunctions(QQ)
s = SF.schur()
m = SF.monomial()

# --- add near the top (we already have s = SF.schur(), so reuse it) ---

def lr_coeff(alpha, beta, lam):
    """
    Littlewood–Richardson coefficient c^{lam}_{alpha, beta}
    using Schur scalar product.
    """
    return ZZ((s(Partition(alpha)) * s(Partition(beta))).scalar(s(Partition(lam))))
# -- keep lr_coeff as is --

def _ell_from_lambda_k(lam, k):
    """
    Compute ell = (|lambda| - k)/2, return None if not a nonnegative integer.
    """
    t = sum(lam) - k
    if t < 0 or t % 2 != 0:
        return None
    return ZZ(t // 2)

def chi_lambda_lr(lam, k):
    """
    LR-sum version with ell derived from (lambda,k):
      ell = (|lambda| - k)/2.
      χ^λ(u-part) = Σ_{r=0..ell} (-1)^r * c^{λ}_{(2ℓ-r, r), (k)}.
    Returns ZZ in {-1,0,1,...} (here typically -1,0,1).
    """
    ell = _ell_from_lambda_k(lam, k)
    if ell is None:
        return ZZ(0)
    total = ZZ(0)
    for r in range(ell+1):
        alpha = (2*ell - r, r) if r > 0 else (2*ell,)
        total += ((-1)**r) * lr_coeff(alpha, (k,), lam)
    return total

def chi_lambda_closed(lam, k):
    """
    Closed-form version for χ^λ(u-part) with the corrected A,B and ℓ=(|λ|-k)/2.

    If len(λ) >= 4, returns 0.
    A = max(λ3, λ2+λ3 - k)
    B = min(λ2, ℓ, 2ℓ - λ2)
    Value = ((-1)^A + (-1)^B)/2 ∈ {-1,0,1}.
    """
    ell = _ell_from_lambda_k(lam, k)
    if ell is None:
        return ZZ(0)

    parts = list(lam) + [0]*max(0, 4 - len(lam))
    lam1, lam2, lam3, lam4 = parts[0], parts[1], parts[2], parts[3]
    if lam4 > 0:
        return ZZ(0)

    A = max(lam3, lam2 + lam3 - k)
    B = min(lam2, ell, 2*ell - lam2)
    if A > B:
        return ZZ(0)
    return ZZ(((-1)**A + (-1)**B) // 2)

def col7_special_u_closed(lam, mu, i, j, k, n_chi):
    """
    Special case j==i, using CLOSED formula with corrected A,B and ℓ=(|λ|-k)/2.
    χ^{λ,μ}(u e_{W_I}) = 0 if |μ| odd; else (-1)^{|μ|/2 + μ1} * χ^λ(u-part).
    """
    mu_size = sum(mu)
    if mu_size % 2 == 1:
        return ZZ(0)
    mu1 = mu[0] if len(mu) > 0 else 0
    phase = (-1)**(mu_size//2 + mu1)
    val_lambda = chi_lambda_closed(lam, k)
    return ZZ(phase * val_lambda)

def col7_special_u_lr(lam, mu, i, j, k, n_chi):
    """
    Special case j==i, LR-SUM version with ℓ=(|λ|-k)/2 to match the closed form.
    """
    mu_size = sum(mu)
    if mu_size % 2 == 1:
        return ZZ(0)
    mu1 = mu[0] if len(mu) > 0 else 0
    phase = (-1)**(mu_size//2 + mu1)
    val_lambda = chi_lambda_lr(lam, k)
    return ZZ(phase * val_lambda)



# ---- Kostka with composition weight (sort to partition, drop zeros)
def kostka_number(lambda_part, weight_tuple):
    mu = tuple(sorted([t for t in weight_tuple if t > 0], reverse=True))
    if sum(mu) != sum(lambda_part):
        return ZZ(0)
    lam = Partition(lambda_part)
    return ZZ(m(s(lam)).coefficient(Partition(mu)))

# ---- κ(·) and f_χ
def kappa(part):
    if not part:
        return ZZ(0)
    lam = Partition(part)
    lc  = lam.conjugate()
    return sum(binomial(x,2) for x in lam) - sum(binomial(y,2) for y in lc)

def reflection_statistic_f(n, lam, mu):
    return ZZ(n*n + (sum(lam) - sum(mu)) + 2*(kappa(lam) + kappa(mu)))

# ---- NEW notation utilities: (i, i+j), n = i + j + k
def parabolic_length_C_new(i, j, k):
    # Components: A_{i-1}, A_{j-1}, C_k
    return (i-1)*i//2 + (j-1)*j//2 + k**2

def multiplicities_bipartitions_new(n, i, j):
    """
    NEW notation: k = n - i - j, with 1 <= i <= j and k >= 0.
    n_χ = sum_{a=0..i} sum_{b=0..j} K_{λ,(a,b,k)} K_{μ,(i-a, j-b)}.
    Returns dict {(λ,μ): n_χ}.
    """
    k = n - i - j
    if k < 0:
        raise ValueError(f"Need n >= i + j; got n={n}, i={i}, j={j}.")
    out = {}
    for a in range(i+1):
        for b in range(j+1):
            n_plus  = a + b + k
            n_minus = (i - a) + (j - b)
            for lam in Partitions(n_plus):
                K1 = kostka_number(lam, (a, b, k))
                if K1 == 0: 
                    continue
                for mu in Partitions(n_minus):
                    K2 = kostka_number(mu, (i-a, j-b))
                    if K2 == 0:
                        continue
                    key = (tuple(lam), tuple(mu))
                    out[key] = out.get(key, ZZ(0)) + K1*K2
    return out

# ---- Column 7 hooks
def col7_generic_w0(lam, mu, n_chi):
    # j != i:  χ^{λ,μ}(w0 e_{W_I}) = (-1)^{|μ|} n_χ
    return ((-1) ** sum(mu)) * n_chi

def col7_special_u_placeholder(lam, mu, i, j, k, n_chi):
    # j == i: plug your χ^{λ,μ}(u e_{W_I}) here (defaults to None)
    return None

# ---- Carter–Geck–Pfeiffer hook–cohook degree d_{λ,μ}(q) in Q[q]
def generic_degree(lam, mu, n):
    """
    Returns d_{λ,μ}(q) in Q[q] using your hook–cohook construction.

    Implementation detail:
      d(q) = q^{a(S)} * Π_{i=1}^n (q^{2i}-1) / ( 2^{b(S)} * H(q) * C(q) )
    We compute H(q)*C(q) as a polynomial in R and perform an **exact quotient**.
    The factor 2^{-b(S)} is applied as a rational scalar (still in Q[q]).
    """
    lam_list = list(lam); mu_list = list(mu)

    k = max(len(lam_list), len(mu_list) + 1)
    lam_pad = lam_list + [0]*(k - len(lam_list))
    mu_pad  = mu_list  + [0]*((k-1) - len(mu_list))

    # X and Y
    X = [lam_pad[i] - (i+1) + k      for i in range(k)]
    Y = [mu_pad[j]  - (j+1) + (k-1)  for j in range(k-1)]

    A = X + Y
    m_len = len(A)

    # a(S)
    s_min = 0
    for u in range(m_len-1):
        Au = A[u]
        for v in range(u+1, m_len):
            s_min += min(Au, A[v])
    s_corr = sum(binomial(m_len - 2*(t+1), 2) for t in range(m_len//2))
    aS = s_min - s_corr

    # b(S)
    from collections import Counter
    cX = Counter(X); cY = Counter(Y)
    inter_min = sum(min(cX[v], cY[v]) for v in set(A))
    bS = (m_len - 1)//2 - inter_min

    # hooks / cohooks
    def hooks_row(row):
        row_set = set(row); pairs = []
        for c in row:
            for b in range(c):
                if b not in row_set:
                    pairs.append((b, c))
        return pairs
    def cohooks_from_to(row, other):
        other_set = set(other); pairs = []
        for c in row:
            for b in range(c):
                if b not in other_set:
                    pairs.append((b, c))
        return pairs

    hooks   = hooks_row(X) + hooks_row(Y)
    cohooks = cohooks_from_to(X, Y) + cohooks_from_to(Y, X)

    # Build numerator in R
    num = R(1)
    for i2 in range(1, n+1):
        num *= (q**(2*i2) - 1)
    num *= q**int(aS)

    # Denominator: polynomial part and scalar 2^{bS}
    den_poly = R(1)
    for (b, c) in hooks:
        den_poly *= (q**(c-b) - 1)
    for (b, c) in cohooks:
        den_poly *= (q**(c-b) + 1)
    scalar = QQ(1) / (QQ(2)**int(bS))   # rational scalar

    # Exact division of polynomials (should divide in R)
    qout, rem = num.quo_rem(den_poly)
    if rem != 0:
        # Fallback (should not happen for correct hook–cohook): raise for visibility
        raise ArithmeticError("generic_degree: polynomial division not exact; got remainder.")
    return scalar * qout   # in Q[q]

# ---- Table builder (NEW notation, 1 <= i <= j)
def table_rows_for_new(n, i, j, degree_fn=None, special_u_fn=None, special_u_version="closed"):
    """
    Build rows for fixed (n, i, j) with initial pair (i, i+j), k = n - i - j.
    Columns: n, (i,i+j), (lambda,mu), f_chi - 2*l(w_I), d_chi, n_chi, col7
      - Generic (j != i): col7 = (-1)^{|μ|} n_χ
      - Special (j == i): choose 'closed' or 'lr' via 'special_u_version',
                          or pass a custom special_u_fn(lam,mu,i,j,k,n_chi)
    """
    if i < 1 or j < i:
        raise ValueError("Require 1 <= i <= j.")
    k = n - i - j
    if k < 0:
        raise ValueError(f"Need n >= i + j; got n={n}, i={i}, j={j}.")
    lWI = parabolic_length_C_new(i, j, k)
    mults = multiplicities_bipartitions_new(n, i, j)

    if degree_fn is None:
        degree_fn = lambda lam, mu: generic_degree(lam, mu, n)

    # choose special-case engine
    if special_u_fn is None and j == i:
        if special_u_version == "lr":
            special_u_fn = col7_special_u_lr
        else:
            special_u_fn = col7_special_u_closed

    records = []
    for (lam, mu), n_chi in sorted(mults.items()):
        fchi = reflection_statistic_f(n, lam, mu)
        dval = degree_fn(lam, mu)                # in Q[q]
        if j != i:
            col7 = col7_generic_w0(lam, mu, n_chi)
        else:
            col7 = special_u_fn(lam, mu, i, j, k, n_chi)
        records.append({
            "n": n,
            "(i,j)": (i, j),
            "(lambda,mu)": (lam, mu),
            "f_chi - 2*l(w_I)": ZZ(fchi - 2*lWI),
            "d_chi": dval,          # Q[q]
            "n_chi": ZZ(n_chi),
            "col7": col7
        })
    return records


# ======= Aggregation and pretty print for 1/Z_C (generic case) =======

def aggregate_factors_1_over_Z(n, i, j):
    """
    For generic c=4 (i != j): group rows by (m, sign) with
      m = (f_chi - 2*lWI)/2, sign s = (-1)^{|mu|}.
    Returns a dict {(m, s): exponent(q) in Q[q]} where
      1/Z_C = Π_{(m,s)} (1 - s*q^m*u^4)^{exponent(m,s)}.
    """
    if j == i:
        raise ValueError("aggregate_factors_1_over_Z assumes generic case j != i (c=4).")
    rows = table_rows_for_new(n, i, j)
    exps = {}
    for r in rows:
        lam, mu = r["(lambda,mu)"]
        m_val = ZZ(r["f_chi - 2*l(w_I)"]) // 2
        sgn   = 1 if (sum(mu) % 2 == 0) else -1
        exp   = r["d_chi"] * ZZ(r["n_chi"])   # stays in Q[q]
        exps[(m_val, sgn)] = exps.get((m_val, sgn), R(0)) + exp
    # (optional) factorization is done at print time
    return exps

def pretty_print_factors(exps):
    """
    Print factors in increasing m, with '+' meaning (1 + q^m u^4) and '-' meaning (1 - q^m u^4).
    Exponents are displayed **factored** in Q[q] (no denominators).
    """
    for (m_val, sgn) in sorted(exps.keys()):
        sign_symbol = '-' if sgn == +1 else '+'
        expo = exps[(m_val, sgn)]
        print(f"(1 {sign_symbol} q^{m_val} u^4)^{expo.factor()}")

# ---------- NEW: pretty factorization helpers and per-row factor listing ----------

def _factored_poly_str(p):
    """Return a nice factored string for a polynomial p in Q[q]."""
    return str(p.factor())

def _expo_pair_str(n_int, poly):
    """Render exponent as [n_chi] * [factored d_chi]."""
    return f"[{ZZ(n_int)}]*[{_factored_poly_str(poly)}]"

def list_factors_per_row(n, i, j, special_u_version="closed"):
    """
    Produce a list of factor-descriptors for each row WITHOUT merging.
    Each item is a dict with keys:
      'base'  : 'u4' or 'u2'   (the power of u appearing),
      'sign'  : +1 or -1       (for u^4 case; for u^2 both signs may appear),
      'power' : integer m so the base is (1 -/+ q^m u^4) or (1 -/+ q^m u^2),
      'exp_n' : integer n_part (e.g. n_chi, or (n_chi ± col7)/2),
      'exp_d' : polynomial d_part in Q[q], i.e., d_chi (already in R),
      'row'   : the original row for reference.
    """
    rows = table_rows_for_new(n, i, j, special_u_version=special_u_version)
    out = []
    # Common quantities
    k = n - i - j
    lWI = parabolic_length_C_new(i, j, k)

    for r in rows:
        lam, mu = r["(lambda,mu)"]
        d_poly  = r["d_chi"]             # Q[q]
        n_chi   = ZZ(r["n_chi"])
        mu_size = sum(mu)

        if j != i:
            # Generic: base is u^4, m = (f - 2*lWI)/2, sign = (-1)^{|mu|}
            m_val = ZZ(r["f_chi - 2*l(w_I)"]) // 2
            sgn   = 1 if (mu_size % 2 == 0) else -1  # +1 => (1 - q^m u^4), -1 => (1 + ...)
            out.append({
                "base": "u4", "sign": sgn, "power": m_val,
                "exp_n": n_chi, "exp_d": d_poly, "row": r
            })
        else:
            # Special case i=j
            # We need f-2*lWI to determine the q-power
            f_minus_2l = ZZ(r["f_chi - 2*l(w_I)"])

            if mu_size % 2 == 1:
                # odd: only (1 + q^{(f-2l)/2} u^4)^{[n_chi]*[d_chi]}
                m_half = f_minus_2l // 2
                out.append({
                    "base": "u4", "sign": -1, "power": m_half,
                    "exp_n": n_chi//2, "exp_d": d_poly, "row": r
                })
            else:
                # even: split into u^2 factors with exponents (n_chi ± col7)/2
                # base power is m_quarter = (f-2l)/4
                m_quarter = f_minus_2l // 4
                col7 = ZZ(r["col7"])
                a = (n_chi + col7) // 2
                b = (n_chi - col7) // 2
                # only emit nonzero exponents:
                if a != 0:
                    # (1 - q^{m_quarter} u^2)^[a]*[d_chi]
                    out.append({
                        "base": "u2", "sign": +1, "power": m_quarter,
                        "exp_n": a, "exp_d": d_poly, "row": r
                    })
                if b != 0:
                    # (1 + q^{m_quarter} u^2)^[b]*[d_chi]
                    out.append({
                        "base": "u2", "sign": -1, "power": m_quarter,
                        "exp_n": b, "exp_d": d_poly, "row": r
                    })
    return out
def pretty_print_factors_per_row(factors):
    """
    Print each row's factor with its (lambda,mu) label:
      (lambda,mu)=(...) : (1 - q^m u^4)^[n]*[d]
    """
    for F in factors:
        base = F["base"]
        sgn  = F["sign"]
        mval = F["power"]
        nexp = F["exp_n"]
        dexp = F["exp_d"]
        lam, mu = F["row"]["(lambda,mu)"]

        # sign symbol and u-power
        if base == "u4":
            sign_symbol = '-' if sgn == +1 else '+'
            factor_str = f"(1 {sign_symbol} q^{mval} u^4)^" + _expo_pair_str(nexp, dexp)
        elif base == "u2":
            sign_symbol = '-' if sgn == +1 else '+'
            factor_str = f"(1 {sign_symbol} q^{mval} u^2)^" + _expo_pair_str(nexp, dexp)
        else:
            raise ValueError("Unknown base type")

        print(f"(lambda,mu)={lam,mu}: {factor_str}")



# ======= Example runs =======
# Generic example: n=4, (i,i+j)=(1,3) -> i=1, j=2, k=1
(n,i,j)=(4,1,2)
facs = list_factors_per_row(n,i,j)     # i=1, j=2
pretty_print_factors_per_row(facs)
rows = table_rows_for_new(n,i,j)
#col7 is chi(ue_{W_I})
rows


(lambda,mu)=((1,), (2, 1)): (1 + q^5 u^4)^[1]*[q^5 * (q^2 + 1)^2 * (q^4 + 1)]
(lambda,mu)=((1,), (3,)): (1 + q^8 u^4)^[1]*[(1/2) * q^2 * (q + 1)^2 * (q^2 - q + 1) * (q^4 + 1)]
(lambda,mu)=((1, 1), (1, 1)): (1 - q^4 u^4)^[1]*[(1/2) * q^6 * (q^2 + 1) * (q^2 + q + 1) * (q^4 + 1)]
(lambda,mu)=((1, 1), (2,)): (1 - q^6 u^4)^[2]*[q^4 * (q^2 - q + 1) * (q^2 + q + 1) * (q^4 + 1)]
(lambda,mu)=((1, 1, 1), (1,)): (1 + q^4 u^4)^[1]*[(1/2) * (q + 1)^2 * q^6 * (q^2 - q + 1) * (q^4 + 1)]
(lambda,mu)=((2,), (1, 1)): (1 - q^6 u^4)^[1]*[(1/2) * q^4 * (q^2 - q + 1) * (q^2 + q + 1) * (q^2 + 1)^2]
(lambda,mu)=((2,), (2,)): (1 - q^8 u^4)^[2]*[(1/2) * q^2 * (q^2 + 1) * (q^2 + q + 1) * (q^4 + 1)]
(lambda,mu)=((2, 1), (1,)): (1 + q^7 u^4)^[3]*[q^3 * (q^2 + 1)^2 * (q^4 + 1)]
(lambda,mu)=((2, 1, 1), ()): (1 - q^6 u^4)^[1]*[(1/2) * q^4 * (q^2 - q + 1) * (q^2 + q + 1) * (q^4 + 1)]
(lambda,mu)=((2, 2), ()): (1 - q^8 u^4)^[1]*[(1/2) * q^2 * (q^2 - q + 1) * (q^2 + 1) * (q^4 + 1)]
(lambda,mu)=((3,), (1,)): (1 + q^10 u^

[{'n': 4,
  '(i,j)': (1, 2),
  '(lambda,mu)': ((1,), (2, 1)),
  'f_chi - 2*l(w_I)': 10,
  'd_chi': q^13 + 2*q^11 + 2*q^9 + 2*q^7 + q^5,
  'n_chi': 1,
  'col7': -1},
 {'n': 4,
  '(i,j)': (1, 2),
  '(lambda,mu)': ((1,), (3,)),
  'f_chi - 2*l(w_I)': 16,
  'd_chi': 1/2*q^10 + 1/2*q^9 + 1/2*q^7 + q^6 + 1/2*q^5 + 1/2*q^3 + 1/2*q^2,
  'n_chi': 1,
  'col7': -1},
 {'n': 4,
  '(i,j)': (1, 2),
  '(lambda,mu)': ((1, 1), (1, 1)),
  'f_chi - 2*l(w_I)': 8,
  'd_chi': 1/2*q^14 + 1/2*q^13 + q^12 + 1/2*q^11 + q^10 + 1/2*q^9 + q^8 + 1/2*q^7 + 1/2*q^6,
  'n_chi': 1,
  'col7': 1},
 {'n': 4,
  '(i,j)': (1, 2),
  '(lambda,mu)': ((1, 1), (2,)),
  'f_chi - 2*l(w_I)': 12,
  'd_chi': q^12 + q^10 + 2*q^8 + q^6 + q^4,
  'n_chi': 2,
  'col7': 2},
 {'n': 4,
  '(i,j)': (1, 2),
  '(lambda,mu)': ((1, 1, 1), (1,)),
  'f_chi - 2*l(w_I)': 8,
  'd_chi': 1/2*q^14 + 1/2*q^13 + 1/2*q^11 + q^10 + 1/2*q^9 + 1/2*q^7 + 1/2*q^6,
  'n_chi': 1,
  'col7': -1},
 {'n': 4,
  '(i,j)': (1, 2),
  '(lambda,mu)': ((2,), (1, 1)),
  'f_chi - 2